## 마침내 완성된 코드 한바닥을 만들었다
> * 나의 첫 완성코드!!!
> * W_REPORT_2011_ver07_yame_ver01.ipynb 에서 맞딱뜨렸던 실력부족을 전부 해결함!!!

In [ ]:
from google.colab import drive
from os.path import join
ROOT = "/content/drive"
print(ROOT)
drive.mount(ROOT)

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 2022년 데이터 작업<<<<<<<<
# 2022년 데이터 작업<<<<<<<<
# 2022년 데이터 작업<<<<<<<<
yyyy = '2022'     #<<<<<<<<<<<<<<< 작업데이터 년도를 내가 입력해야됨!!!!!
# 2022년 데이터 작업<<<<<<<<
# 2022년 데이터 작업<<<<<<<<
# 2022년 데이터 작업<<<<<<<<


## 코랩을 사용할 때
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive/')

#현재 작업 위치 이동
#띄어쓰기에 \붙일 것
%cd /content/drive/MyDrive/Dev_work/REPORT



import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Dev_work/REPORT


## 이렇게 빨리 다시 리스트 기능을 복습하게 될 줄이야 (22 1103 22:47)
> * 모델 돌려본 결과 때문에 프젝 방향을 크게 틀어서
>> * 기존 : 특보 발효시점 예측과 해제시점 예측
>> * 방향전환 : 발효 후를 학습시켜 해제시점만 예측
> * 고로 해제시점=특보강도=0 만이 아니라, **발효직전시점=특보강도=0도 필요**하게 되었음
> * 호딱끝내자~~~~~!
> * 그리고 빨리 끝나면 해양기상 관련 논문을 좀 찾아봐둬도 좋겠다.
> * 솔직히 프론트엔드보다는 모델 공부 하고 싶지만 지금 시간이 너무 없어서... 차라리 그게 지금 팀프젝에 도움 되지 싶음

In [ ]:
df = pd.read_csv(   '/content/drive/MyDrive/Dev_work/REPORT/DATA_csv/기상특보_W_REPORT/코드정리만 된 것으로 뽑아낸 csv/W_REPORT_' + yyyy + '_yame.csv')
# /content/drive/MyDrive/Dev_work/REPORT/DATA_csv/기상특보_W_REPORT/코드정리만 된 것으로 뽑아낸 csv/W_REPORT_2011_yame.csv
df

,일시,특보강도
0,2022-01-11 12:00:00,1
1,2022-01-12 08:00:00,1
2,2022-01-12 09:00:00,0
3,2022-01-20 12:00:00,1
4,2022-01-20 20:00:00,1
...,...,...
63,2022-09-19 15:00:00,2
64,2022-09-20 07:00:00,2
65,2022-09-20 08:00:00,1
66,2022-09-21 10:00:00,1


In [ ]:
# ver01에서 먹혔던
# 손미란 강사님의 프로세스:
  # 
  # 결측치 리스트 추출
  # 결측치의 인덱스 추출
  # 해당 인덱스 +1의 인덱스 추출
  #                 ㄴ-인덱스의 발효시점데이터값 추출
  #                 +데이터 +1hour 연산
  #                 해당 값을 결측치에 할당
  # ----결측치 리스트만큼 반복하기. 각각 for문을 이용해서.


  # 1  ndf['특보강도']='0' 의 리스트 추출
  # 2  ㄴ-의 인덱스만 추출
  # 3  ㄴ-의 인덱스+1의 인덱스만 추출 (=새 특보발효가 터지는 순간임)
  # 4  ㄴ-의 ['일시'] -dt.timedelta(hour=1) 연산
  # 5  ㄴ-해당 값을 새 newTime_Ndf에 append
  # 6  ㄴ-이걸 for문 이용해서 반복하기.                                                          # 나는 append를 써볼게
# 7  그리고 마무리: 첫 특보의 특보발효직전시점=첫특보발효-1hour(특보lv=0)부터 넣고 인덱스 재정렬  <= concat, append, loc 으로 데이터프레임 행 추가 레퍼 https://zephyrus1111.tistory.com/64


In [ ]:
ndf = df
ndf.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일시      68 non-null     object
 1   특보강도    68 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


In [ ]:
# 왜 ['일시']가 object지? datetime이어야하는데
ndf['일시'] = pd.to_datetime(ndf['일시'])
ndf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      68 non-null     datetime64[ns]
 1   특보강도    68 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.2 KB


In [ ]:
# 일단 작업하기 편하게 기존 인덱스인 datetime을 새 컬럼으로 보내고 01234로 이루어진 새 인덱스를 만들어주자
# set_index 기능과 reset_index 기능 레퍼 https://datascienceschool.net/01%20python/04.05%20%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%94%84%EB%A0%88%EC%9E%84%20%EC%9D%B8%EB%8D%B1%EC%8A%A4%20%EC%A1%B0%EC%9E%91.html
ndf.reset_index(drop=False) #<=drop=False가 기본값인데, 이렇게 하면 기존 인덱스가 컬럼으로 유지됨
ndf.head()

,일시,특보강도
0,2022-01-11 12:00:00,1
1,2022-01-12 08:00:00,1
2,2022-01-12 09:00:00,0
3,2022-01-20 12:00:00,1
4,2022-01-20 20:00:00,1


In [ ]:
  # 1  ndf['특보강도']='0' 의 리스트 추출
LvZero_cave = ndf[ndf['특보강도']==0]
# LvZero_cave.index[0]                  #<=잘 나오는거 확인함


  # 2  ㄴ-의 인덱스만 추출                  # 데이터프레임 인덱스를 리스트로 변환하기 레퍼 https://aplab.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%94%84%EB%A0%88%EC%9E%84-%EB%A6%AC%EC%8A%A4%ED%8A%B8
LvZero_caveIDX_list = LvZero_cave.index.to_list()
# LvZero_caveIDX_list[10]               #<= 34 나옴. 잘 나오는거 확인함!

# len(LvZero_caveIDX_list_nextIDX)    #<=39개
# LvZero_caveIDX_list_nextIDX[ len(LvZero_caveIDX_list_nextIDX)-1 ]   #<=리스트의 맨 마지막 요소 131


  # 3  ㄴ-의 인덱스+1의 인덱스만 추출 (=새 특보발효가 터지는 시점임. 있다가 여기에 +1hours 해준걸 새 행으로 append 해줘야함)
LvZero_caveIDX_list_nextIDX = []
minus_1 = len(LvZero_caveIDX_list)-1  #<= indexer out of bound 에러에 대한 임시방편
i=0
for i in range(minus_1):
    LvZero_caveIDX_list_nextIDX.append( LvZero_caveIDX_list[i] +1 )
    i+1
# LvZero_caveIDX_list_nextIDX             #<= +1된걸로 주루룩 나옴. 잘 나오는거 확인함! 와 미쳤다 yame_ver01에서 못했던거 지금 다 찾아내는중; 실력 자체는 안 늘지만 레퍼는 늘어난다 나중에 이거 다 복습하면 대박사건; 프젝 끝나고 조원분들에게도 가르쳐드리면 짱이겟는데
# ======잘 나오는건 괜찮은데, 4번 항목에서 for문 돌리려니 지금 자꾸 indexer out of bound 에러가 뜸. 
# 임시방편으로 minus_1 이라는 변수 하나 만들어서 범위 대체했다 ㅋㅋㅋㅋㅋ
    # expected an indented block 에러가 3번에서 자꾸 반복되길래 들여쓰기 한번 탭해주니 작동함...허무함

  # 4  ㄴ-의 요소를 인덱스 삼아, ndf['일시'] -dt.timedelta(hours=1) 연산
# 이거는 그냥 새 df를 만들자: df명은 mdf
# 이 mdf에 df명.append 해서 -dt.timedelta 연산결과를 추가해주자. 그리고 나중에 ndf에 콘캣 시키는거지
mdf = pd.DataFrame(data=None, index=None, dtype=None, copy=False)
i=0
for i in range(minus_1):
    new_time_date_dictio = {
      '일시' : ndf.iloc[ LvZero_caveIDX_list_nextIDX[i],0 ] - dt.timedelta(hours=1),
      '특보강도' : 0
    }
    i+1
    mdf = mdf.append(new_time_date_dictio, ignore_index=True)
# new_time_date_dictio    #<= {'일시': Timestamp('2011-12-08 20:00:00'), '특보강도': 0} 출력됨. 마지막까지 잘 만들어냈다.
# mdf   #<=전부 잘 추가된 것 확인함



  # 5  ㄴ-해당 값을 새 newTime_Ndf에 append
newTime_Ndf = ndf
newTime_Ndf = newTime_Ndf.append(mdf, ignore_index=True)
# newTime_Ndf



  # 6  ㄴ-이걸 for문 이용해서 반복하기.                                                          # 나는 append를 써볼게
# ==>해냄



# 7  그리고 마무리: 첫 특보의 특보발효직전시점=첫특보발효-1hour(특보lv=0)부터 넣고 인덱스 재정렬  <= concat, append, loc 으로 데이터프레임 행 추가 레퍼 https://zephyrus1111.tistory.com/64
  #<= 131	2011-01-01 15:00:00	0 가 잘 들어갔다. 이건 나중에 ndf['일시']를 다시 set_index 한 후 정렬하면 됨
first_Lv0_point = {
    '일시' : ndf.iloc[0,0] - dt.timedelta(hours=1),  #<=항상 이 끝 콤마가 중요하다
    '특보강도':0
    }
newTime_Ndf = newTime_Ndf.append(first_Lv0_point, ignore_index=True)
# newTime_Ndf.tail()

newTime_Ndf = newTime_Ndf.set_index('일시', drop=True)
newTime_Ndf = newTime_Ndf.sort_index(ascending=True)    # # 인덱스 기준으로 행 재정렬   레퍼 https://hogni.tistory.com/6
newTime_Ndf

#==========끝 이제 저장만 하면 됩니다


,특보강도
일시,
2022-01-11 11:00:00,0.0
2022-01-11 12:00:00,1.0
2022-01-12 08:00:00,1.0
2022-01-12 09:00:00,0.0
2022-01-20 11:00:00,0.0
...,...
2022-09-19 15:00:00,2.0
2022-09-20 07:00:00,2.0
2022-09-20 08:00:00,1.0


In [ ]:
newTime_Ndf = newTime_Ndf.to_csv(  '/content/drive/MyDrive/Dev_work/REPORT/DATA_csv/기상특보_W_REPORT/코드정리만 된 것으로 뽑아낸 csv/W_REPORT_'+yyyy+'_ver07_02__first_Lv0_point.csv'  , index='False', encoding='UTF-8')